# Initial ETL

In this notebook I am gonna do a bunch of stuff to get the data ready for the model. I might need to come back and do some more ETL as I work on the modeling bit, but this is the initial round.

In [1]:
%matplotlib notebook
import pandas as pd
import numpy as np
from utils.utilities import mk_heatmap, value_heatmap, select_by_date, update_grade
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn import preprocessing

In [2]:
# Load all the data
inspecs = pd.read_csv('./data/inspections.csv')
threeoneone = pd.read_csv('./data/311.csv')
nypd = pd.read_csv('./data/nypd.csv')
weather = pd.read_csv('./data/NYC_historical_weather.csv')

### But there are multiple citations for each restaurant in the df

For some reason I missed the fact that each row in the inspections data contains a single violation. We are gonna need to merge all the different rows into single restaurant inspections.

In [3]:
inspecs_single_visits = inspecs.drop_duplicates(subset=['camis', 'inspection_date'])

cols_to_merge = ['violation_code', 'violation_description', 'critical_flag', 'inspection_type']

for col in cols_to_merge:
    df_tmp = inspecs[col].groupby([inspecs.camis, inspecs.inspection_date]).apply(list).reset_index()
    inspecs_single_visits = pd.merge(inspecs_single_visits, df_tmp,  how='left', left_on=['camis','inspection_date'], right_on = ['camis','inspection_date'])
    inspecs_single_visits.drop(columns=f'{col}_x', axis=1, inplace=True)
    inspecs_single_visits.rename(columns={f'{col}_y': f"{col}"}, inplace=True)

In [4]:
inspecs_single_visits.shape

(82569, 15)

In [5]:
inspecs = inspecs_single_visits

# Clean up some of the dtypes and replace missing values with better values

In [6]:
# Replace the missing lats and lons with nan
inspecs['latitude'].replace(0, np.nan, inplace=True)
inspecs['longitude'].replace(0, np.nan, inplace=True)
threeoneone['latitude'].replace(0, np.nan, inplace=True)
inspecs['longitude'].replace(0, np.nan, inplace=True)
nypd['latitude'].replace(0, np.nan, inplace=True)
inspecs['longitude'].replace(0, np.nan, inplace=True)

# drop rows that don't have a lat and lon attached
inspecs.dropna(subset = ['latitude', 'longitude'], inplace=True)
threeoneone.dropna(subset = ['latitude', 'longitude'], inplace=True)
nypd.dropna(subset = ['latitude', 'longitude'], inplace=True)

# Do date things
inspecs.inspection_date = pd.to_datetime(inspecs.inspection_date)
threeoneone.created_date = pd.to_datetime(threeoneone.created_date)
nypd.complaint_date = pd.to_datetime(nypd.complaint_date)
weather.DATE = pd.to_datetime(weather.DATE)

# update the grades where a score is given but the letter grade is missing
inspecs.grade = inspecs.apply(update_grade, axis=1)


# Merge (or create) extra data into the inspections frame

In [7]:
# Does the inspection have a critical violation?
nested = inspecs['critical_flag'].values.tolist()
inspecs['critical'] = [1 if 'Y' in sublist else 0 for sublist in nested]

# make a column to store which day of the week it is... 
# Remember that Monday is 0 Sunday = 6
inspecs['weekday'] = inspecs.inspection_date.dt.weekday
threeoneone['weekday'] = threeoneone.created_date.dt.weekday
nypd['weekday'] = nypd.complaint_date.dt.weekday

# merge the weather data into the frame
inspecs = inspecs.merge(weather[['DATE', 'TMAX']], left_on='inspection_date', right_on='DATE')
# drop the extra date column
inspecs.drop(labels='DATE', axis=1, inplace=True)
# rename to be lower case
inspecs.rename(columns={'TMAX': 'tmax'}, inplace=True)

# now we are gonnna create a bunch of temporal stuff
# time since last inspection
inspecs['time_since_last'] = inspecs.groupby('camis')['inspection_date'].diff().apply(lambda x: x.days)
# past critical violation?
inspecs['past_critical'] = inspecs.groupby(['camis'])['critical'].shift()
# past grade and score
inspecs['past_grade'] = inspecs.groupby(['camis'])['grade'].shift()
inspecs['past_score'] = inspecs.groupby(['camis'])['score'].shift()


# start_date = '2016-01-01'
# end_date = '2016-12-31'

# inspecs_2016 = select_by_date(inspecs, start_date, end_date)
# threeoneone_2016 = select_by_date(threeoneone, start_date, end_date)
# nypd_2016 = select_by_date(nypd, start_date, end_date)

In [8]:
inspecs.head(5)

,camis,dba,boro,zipcode,cuisine_description,inspection_date,action,score,grade,latitude,...,violation_description,critical_flag,inspection_type,critical,weekday,tmax,time_since_last,past_critical,past_grade,past_score
0,41471993,NEW MEI MEI CHINESE RESTAURANT,Brooklyn,11229.0,Chinese,2016-01-04,Violations were cited in the following area(s).,6.0,A,40.597078,...,[Evidence of mice or live mice present in faci...,[Y],[Cycle Inspection / Re-inspection],1,0,36,NaN,NaN,NaN,NaN
1,50016849,WIBAR,Queens,11369.0,American,2016-01-05,Violations were cited in the following area(s).,9.0,A,40.774414,...,[Non-food contact surface improperly construct...,"[N, Y]","[Cycle Inspection / Initial Inspection, Cycle ...",1,1,29,NaN,NaN,NaN,NaN
2,40670353,DUKE'S,Manhattan,10016.0,American,2016-01-05,Violations were cited in the following area(s).,7.0,A,40.747683,...,[Food not protected from potential source of c...,"[Y, N]","[Cycle Inspection / Initial Inspection, Cycle ...",1,1,29,NaN,NaN,NaN,NaN
3,40585345,SUSHIYA JAPANESE RESTAURANT,Manhattan,10019.0,Japanese,2016-01-05,Violations were cited in the following area(s).,17.0,B,40.762738,...,[Food worker does not use proper utensil to el...,"[Y, N, Y]","[Cycle Inspection / Initial Inspection, Cycle ...",1,1,29,NaN,NaN,NaN,NaN
4,41685208,PAPA'S HALAL CHICKEN & GRILL,Staten Island,10303.0,Indian,2016-01-06,Violations were cited in the following area(s).,13.0,A,40.625071,...,[Food contact surface not properly maintained....,"[N, N, N, Y]","[Cycle Inspection / Re-inspection, Cycle Inspe...",1,2,41,NaN,NaN,NaN,NaN


In [ ]:
# add the 311 and nypd scores to the inspections frame

In [ ]:
inspecs_2016['311_score'] = 0
inspecs_2016['nypd_score'] = 0

for month in range(1, 13):
    df_311_month = threeoneone_2016[threeoneone['created_date'].dt.month == month]
    df_nypd_month = nypd_2016[nypd_2016['complaint_date'].dt.month == month]
    df_inspec_month = inspecs_2016[inspecs_2016['inspection_date'].dt.month == month]
    
    if not df_nypd_month.shape[0]:
        continue
    elif not df_311_month.shape[0]:
        continue
    
    # build the heataps for the 311 and nypd
    s = 2 # sigmas
    bins = 1000
    img_311, extent_311, xedges_311, yedges_311 = mk_heatmap(df_311_month.longitude, 
                                                             df_311_month.latitude, s, bins=bins)
    img_nypd, extent_nypd, xedges_nypd, yedges_nypd = mk_heatmap(df_nypd_month.longitude, 
                                                             df_nypd_month.latitude, s, bins=bins)
    
    for i, row in df_inspec_month.iterrows():
        inspecs_2016.loc[i, '311_score'] = value_heatmap(row.longitude, 
                                                          row.latitude, 
                                                          xedges_311, 
                                                          yedges_311, 
                                                          img_311)
        inspecs_2016.loc[i, 'nypd_score'] = value_heatmap(row.longitude, 
                                                          row.latitude, 
                                                          xedges_nypd, 
                                                          yedges_nypd, 
                                                          img_nypd)

    

In [ ]:
inspecs_2016.info()

In [ ]:
# update the null values in critical flag to 'N'
inspecs_2016.critical_flag.replace(np.nan, 'N', inplace=True)

In [ ]:
# convert some of the catagorical data into numerical data
enc = preprocessing.OrdinalEncoder()

boros = inspecs_2016.boro.unique()

enc.fit(boros.reshape(-1,1))

inspecs_2016['boro_enc'] = enc.transform(inspecs_2016.boro.values.reshape(-1, 1)).flatten()

crit_flags = inspecs_2016.critical_flag.unique()

enc.fit(crit_flags.reshape(-1,1))

inspecs_2016['crit_flag_enc'] = enc.transform(inspecs_2016.critical_flag.values.reshape(-1, 1)).flatten()

In [ ]:
inspecs_2016.to_csv('./data/inspecs_2016_map.csv', index=False)

In [ ]:
inspecs_2016[inspecs_2016.critical_flag.isnull()].iloc[0]

In [ ]:
df = nypd_2016

In [ ]:
month_hist = df.complaint_date.dt.month
month_hist = month_hist.value_counts().sort_index()
month_hist.index = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
plt.bar(month_hist.index, month_hist.values)
plt.xlabel("Month")
plt.ylabel("Count of Inspections")

In [ ]:
month_hist

In [ ]:
day_hist = pd.to_datetime(df["inspection_date"]).dt.weekday
day_hist = day_hist.value_counts().sort_index()
day_hist.index = ["Mon", "Tue", "Wed", "Thur", "Fri", "Sat", "Sun"]
plt.bar(day_hist.index, day_hist.values)
plt.xlabel("Day of Week")
plt.ylabel("Count of Inspections")

In [ ]:
df[df.inspection_date.dt.month == 1][['latitude', 'longitude']]

In [ ]:
score_heatmap_month(y=40.597078, x=-73.941255, df=df, month=0)

In [ ]:
fig, ax = plt.subplots(1,1)

x = df.longitude[df.inspection_date.dt.month == 1]
y = df.latitude[df.inspection_date.dt.month == 1]

s = 2

img, extent, xedges, yedges = mk_heatmap(x, y, s, bins=1000)
ax.imshow(img, extent=extent, origin='lower', cmap=cm.PuRd)
ax.set_title("NYPD  $\sigma$ = %d" % s)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')